You can write notes & updates here:

.
.



In [14]:
#loading libraries & data files
library(tidyverse)
library(tidymodels)
library(ggplot2)

players_data <- read_csv("https://raw.githubusercontent.com/achan919/dsci-final-project/refs/heads/main/players.csv")
sessions_data <- read_csv("https://raw.githubusercontent.com/achan919/dsci-final-project/refs/heads/main/sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [15]:
#tidying data
#tidy data for players data
players_data <- mutate(players_data, experience = as.factor(experience))
players_data <- mutate(players_data, gender = as.factor(gender))

# tidy date data
sessions_data_time <- sessions_data |>
    mutate(original_end_time = as.numeric(as.POSIXct(end_time, format = "%d/%m/%Y %H:%M")),
           original_start_time = as.numeric(as.POSIXct(start_time, format = "%d/%m/%Y %H:%M")))

# calculate session length from start and end time (in seconds)
sessions_data_plustime <- sessions_data_time |>
  mutate(time = original_end_time - original_start_time)

# convert to numeric diff_time
sessions_data_numeric_time <- sessions_data_plustime |>
  mutate(diff_time = as.numeric(time))

#actual tidy data for use
sessions_data_tidy <- select(sessions_data_numeric_time, hashedEmail, start_time, end_time, diff_time)

Merging both players and sessions data sets 

In [11]:
#merged all data between sessions and tidy data
merged_all_data <- merge(sessions_data_tidy, players_data, by = "hashedEmail", all = T)

#merged data for people with at least one session
merged_sessions_data <- filter(merged_all_data,!is.na(start_time))

#data for people with no sessions (people in players data, but not sessions data)
merged_nosessions_data <- filter(merged_all_data, is.na(start_time))
head(merged_sessions_data)

,hashedEmail,start_time,end_time,diff_time,experience,subscribe,played_hours,name,gender,Age
,<chr>,<chr>,<chr>,<dbl>,<fct>,<lgl>,<dbl>,<chr>,<fct>,<dbl>
1,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc93355ccc95b45423367832,23/05/2024 00:22,23/05/2024 01:07,2700,Regular,TRUE,1.5,Isaac,Male,20
2,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc93355ccc95b45423367832,22/05/2024 23:12,23/05/2024 00:13,3660,Regular,TRUE,1.5,Isaac,Male,20
3,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe8e1cf0eee9a7b67967,28/06/2024 04:28,28/06/2024 04:58,1800,Pro,FALSE,0.4,Lyra,Male,21
4,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce0290fa0437ce0b97f387,19/09/2024 21:01,19/09/2024 21:12,660,Beginner,TRUE,0.1,Osiris,Male,17
5,0d4d71be33e2bc7266ee4983002bd930f69d304288a8663529c875f40f1750f3,31/08/2024 22:44,31/08/2024 23:20,2160,Regular,TRUE,5.6,Winslow,Male,17
6,0d4d71be33e2bc7266ee4983002bd930f69d304288a8663529c875f40f1750f3,01/09/2024 21:59,01/09/2024 22:36,2220,Regular,TRUE,5.6,Winslow,Male,17


tidying sessions data so we know total number of sessions and total time spent playing for each player by name

In [13]:
#tidying sessions
sessions_names_only <- select(merged_sessions_data, name, diff_time)

#making new variables number of sessions, total time, mean time & median time
sessions_count <- sessions_names_only |> count(name) 
total_time <- aggregate(.~name, data = sessions_names_only, FUN=sum)
total_time_summary_stats <- sessions_names_only|>
    group_by(name) |>
    summarize(mean_time= mean(diff_time),median_time= median(diff_time))

#only the variables name, sessions_num, total_time,mean_time, median_time
sessions_by_name<- bind_cols(sessions_count, total_time,total_time_summary_stats )|> 
#head(sessions_by_name)  -- use this to confirm all names match up
    mutate(sessions_num=n, total_time=diff_time,name=name...1)|>
    select(name,sessions_num, total_time,mean_time,median_time)

#players data for all players with sessions with additional variable number of sessions, total time, mean time & median time
sessions_allvars <- merge(sessions_by_name, players_data, by = "name", all.x = T)
head(sessions_allvars)

New names:
• `name` -> `name...1`
• `name` -> `name...3`
• `name` -> `name...5`


,name,sessions_num,total_time,mean_time,median_time,experience,subscribe,hashedEmail,played_hours,gender,Age
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<fct>,<lgl>,<chr>,<dbl>,<fct>,<dbl>
1,Aarav,1,1080,1080.000,1080,Beginner,TRUE,aea049eaa7cb10db386a62990220d205ceb2a4c473cae3ec7bab107278c26559,0.2,Prefer not to say,17
2,Aaron,2,5160,2580.000,2580,Amateur,TRUE,22a78bfd44e46931261d06864b92cf9a8fd77fb02bec66cb71646dd9da222a0d,1.2,Non-binary,17
3,Ahmed,1,900,900.000,900,Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Other,NA
4,Aiden,6,7080,1180.000,900,Veteran,FALSE,88247d9a46fc214a12485dcbcbb03a8ddebfe8c1ec5fe209cd660147625c8e62,1.4,Prefer not to say,25
5,Akio,95,847920,NA,NA,Regular,TRUE,b622593d2ef8b337dc554acb307d04a88114f2bf453b18fb5d2c80052aeb2319,218.1,Non-binary,20
6,Alex,130,232380,1787.538,1320,Amateur,TRUE,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,53.9,Male,17
